In [2]:
from pathlib import Path
import mdtraj as md
from openmm.app import PDBFile

In [19]:
import mdtraj as md

# Load trajectory and topology
traj = md.load_dcd("/home/illiasavchenko/Pavlo/ASP-ARG-ASP/traj.dcd",
                   top="/home/illiasavchenko/Pavlo/ASP-ARG-ASP/topology.pdb")

# Get atom indices for protein (chain A) and peptide (chain B)
protein_atoms = traj.topology.select("chainid == 0")  # chain A
peptide_atoms = traj.topology.select("chainid == 1")  # chain B
solvent_atoms = traj.topology.select("chainid == 2")  # chain B


dcdplugin) detected standard 32-bit DCD file of native endianness
dcdplugin) CHARMM format DCD file (also NAMD 2.1 and later)


In [20]:
from openmm.app import PDBFile, ForceField, PME, HBonds, NoCutoff
from openmm import Platform, Context, LangevinMiddleIntegrator
from openmm.unit import *

# Load system using PDB and force field
pdb = PDBFile("/home/illiasavchenko/Pavlo/ASP-ARG-ASP/topology.pdb")
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")  # adjust as needed
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer, constraints=HBonds)


In [21]:
from openmm.openmm import NonbondedForce

for force in system.getForces():
    if isinstance(force, NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("protein_scale", 1)
        force.addGlobalParameter("peptide_scale", 1)
        force.addGlobalParameter("solvent_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            if i in protein_atoms:
                param = "protein_scale"
            elif i in peptide_atoms:
                param = "peptide_scale"
            else:
                param = "solvent_scale"

            force.setParticleParameters(i, 0, 0, 0)
            force.addParticleParameterOffset(param, i, charge, sigma, epsilon)
            
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)


In [22]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1 / picosecond, 2 * femtoseconds)
platform = Platform.getPlatformByName("CUDA")  # Use CUDA or CPU for speed
context = Context(system, integrator, platform)

In [23]:
def interaction_energy(protein_scale, peptide_scale, solvent_scale):
    context.setParameter("protein_scale", protein_scale)
    context.setParameter("peptide_scale", peptide_scale)
    context.setParameter("solvent_scale", solvent_scale)
    state = context.getState(getEnergy=True, groups={0})
    return state.getPotentialEnergy()


In [26]:
energies = []

for i, frame in enumerate(traj):

    positions = frame.xyz[0] * nanometer
    context.setPositions(positions)

    total = interaction_energy(1, 1, 1)
    protein_only = interaction_energy(1, 0, 0)
    peptide_only = interaction_energy(0, 1, 0)
    solvent_only = interaction_energy(0, 0, 1)
    peptide_solvent = interaction_energy(0, 1, 1) - peptide_only - solvent_only
    protein_solvent = interaction_energy(1, 0, 1) - protein_only - solvent_only
    peptide_protein = interaction_energy(1, 1, 0) - protein_only - peptide_only

    sum_ = sum([protein_only,peptide_only,solvent_only,peptide_solvent,protein_solvent,peptide_protein])
    energies.append([protein_only,peptide_only,solvent_only,peptide_solvent,protein_solvent,peptide_protein])

    # Interaction = total - (protein-only + peptide-only)
    #interaction = total - protein_only - peptide_only
    print(f"Frame {i}: peptide_protein: {peptide_protein}, peptide_solvent: {peptide_solvent}")
    energies.append(interaction)


Frame 0: peptide_protein: -1981.1856956481934 kJ/mol, peptide_solvent: -1278.1806640625 kJ/mol
Frame 1: peptide_protein: -1968.2215042114258 kJ/mol, peptide_solvent: -1459.4755630493164 kJ/mol
Frame 2: peptide_protein: -1918.796802520752 kJ/mol, peptide_solvent: -1246.376205444336 kJ/mol
Frame 3: peptide_protein: -1825.3288612365723 kJ/mol, peptide_solvent: -1284.7447967529297 kJ/mol
Frame 4: peptide_protein: -1894.4516220092773 kJ/mol, peptide_solvent: -1516.3961639404297 kJ/mol
Frame 5: peptide_protein: -1916.6167259216309 kJ/mol, peptide_solvent: -1586.7005081176758 kJ/mol
Frame 6: peptide_protein: -1890.5580978393555 kJ/mol, peptide_solvent: -1308.5420989990234 kJ/mol
Frame 7: peptide_protein: -1835.6193008422852 kJ/mol, peptide_solvent: -1288.7407684326172 kJ/mol
Frame 8: peptide_protein: -1831.083911895752 kJ/mol, peptide_solvent: -1400.9693908691406 kJ/mol
Frame 9: peptide_protein: -1827.281810760498 kJ/mol, peptide_solvent: -1245.829444885254 kJ/mol
Frame 10: peptide_protein: -